<!-- PROJECT LOGO -->
<br/>
<p align="center">
    <img src="https://raw.githubusercontent.com/Team-17-Bedu/proyecto_python/main/img/icono.png" alt="Logo" width="135" height="135">

  <h3 align="center"><strong>Países con mayor calidad de vida</strong></h3>

  <p align="center">
    Proyecto final para el modulo "Procesamiento de datos con Python"
  </p>
</p>

#### Acerca del proyecto:

Este proyecto tiene como proposito realizar un analisis de la calidad de vida de ciertos paises, con el fin de determinar cuál de ellos podria ser la mejor opcion para vivir o buscar un empleo, todo esto, haciendo uso de herramientas estadísticas y computacionales, siendo el principal aliado, el lenguaje de programacion Python. 


En este caso se tendra como principal objetivo en calcular el IDH por medio de la documentación proporcionada del __*Human Development Report 2020*__, presente en el siguiente [enlace](http://hdr.undp.org/sites/default/files/hdr2020_technical_notes.pdf). Y con tales resultados justificar nuestro anterior proyecto elaborado en el lenguaje R, el proyecto esta presente en el siguiente [enlace](https://github.com/Team-17-Bedu/proyecto).

Todos los datos serán obtenidos por medio de la API proporcionada por la UNDP

#### Procedimiento:
Para comenzar se importaron los modulos necesarios (dadas las limitaciones de github no es posible renderizar los widgets, para ello sera necesario visualuzar el notebook en Google Colab)


In [92]:
import pandas as pd
import requests
import ipywidgets as widgets
from IPython.display import display

Debido a que el acceso a la API requiere un inicio de sesion, se recurrió al objeto `Session` del modulo `requests`  

In [93]:
session = requests.Session()

Mediante el modulo `widgets` se definieron dos text box para introducir los datos de autenticación, `user` y `password` respectivamente 

In [94]:
user = widgets.Text()
password = widgets.Text()
display(user, password)

Text(value='')

Text(value='')

Definición de la tupla que contendra los datos de acceso

In [96]:
access = (user.value, password.value)

Inicio de sesión en la API de la UNDP

In [97]:
session.get("http://ec2-54-174-131-205.compute-1.amazonaws.com/API/Login.php",
            auth=access)

<Response [200]>

Dada la estructura del `json` recibido, se construyo una funcion para normalizar los datos, a modo de obtener un `DataFrame` homogeneo

In [118]:
def normalize_data(data_dirt: dict, indicator_id: str) -> dict:
    new_data = {
        'Country': list(data_dirt['country_name'].values())
    }
    countries = list(data_dirt['country_name'].keys())
    for year, year_data in data_dirt['indicator_value'].items():
        values = []
        last_key = ''
        for key, val in year_data[indicator_id].items():
            current_pos = countries.index(key)
            last_pos = countries.index(last_key) if last_key != '' else 0
            last_key = key
            zeros = current_pos - last_pos
            i = 1
            while i < zeros and zeros != 0:
                values.append(0)
                i += 1
            values.append(val)
        new_data[year] = values
    return  new_data

Definición del indicador a consultar

In [119]:
indicator = '137506'

Realización de la consulta (puesto que la estructura es diferente a la de otras peticiones, esta se realizo con un `fstring`)

In [120]:
data = session.get(f'http://ec2-54-174-131-205.compute-1.amazonaws.com/API/HDRO_API.php/indicator_id={indicator}/structure=yic')

Normalizacion del `json` obtenido en la consulta

In [121]:
clean = normalize_data(data.json(), indicator)

Creación del `DataFrame` con los datos normalizados

In [123]:
df = pd.DataFrame(clean)
df

,Country,1990,1991,1992,1993,1994,1995,1996,1997,1998,...,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,Afghanistan,0.302,0.307,0.316,0.312,0.307,0.331,0.335,0.339,0.344,...,0.472,0.477,0.489,0.496,0.500,0.500,0.502,0.506,0.509,0.511
1,Angola,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.517,0.533,0.544,0.555,0.565,0.572,0.578,0.582,0.582,0.581
2,Albania,0.650,0.631,0.615,0.618,0.624,0.637,0.646,0.645,0.655,...,0.745,0.764,0.775,0.782,0.787,0.788,0.788,0.790,0.792,0.795
3,Andorra,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.837,0.836,0.858,0.856,0.863,0.862,0.866,0.863,0.867,0.868
4,United Arab Emirates,0.723,0.735,0.738,0.745,0.755,0.764,0.765,0.766,0.769,...,0.820,0.826,0.832,0.838,0.847,0.859,0.864,0.881,0.889,0.890
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
184,Samoa,0.633,0.634,0.634,0.639,0.618,0.624,0.633,0.635,0.641,...,0.698,0.701,0.698,0.700,0.703,0.707,0.710,0.710,0.709,0.715
185,Yemen,0.401,0.401,0.404,0.406,0.408,0.414,0.421,0.426,0.431,...,0.506,0.506,0.504,0.509,0.502,0.483,0.474,0.467,0.468,0.470
186,South Africa,0.627,0.634,0.642,0.648,0.650,0.653,0.650,0.646,0.640,...,0.664,0.665,0.675,0.685,0.693,0.701,0.703,0.705,0.707,0.709
187,Zambia,0.421,0.417,0.416,0.419,0.414,0.415,0.416,0.416,0.416,...,0.527,0.534,0.549,0.557,0.561,0.569,0.571,0.578,0.582,0.584
